# Shuttle

In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import fbeta_score, roc_auc_score, average_precision_score, matthews_corrcoef
import numpy as np
from collections import defaultdict

# Load the training dataset
train_data = pd.read_csv("shuttle_train.csv")
train_data['label'] = train_data['label'] - 1  # Subtract 1 to make labels zero-based

# Load the test dataset
test_data = pd.read_csv("shuttle_test.csv")
test_data['label'] = test_data['label'] - 1  # Subtract 1 to make labels zero-based

# Parameters
num_runs = 5  # Number of iterations
num_round = 100
params = {
    "objective": "multi:softprob",  # Output probabilities
    "num_class": len(train_data['label'].unique()),  # Number of unique classes in the target
    "eval_metric": "mlogloss",
    "max_depth": 6,
    "eta": 0.3,
    "seed": 42
}

# Initialize accumulators
accuracy_scores = []
f2_scores = []
auc_roc_scores = []
auc_pr_scores = []
mcc_scores = []

# Initialize metrics accumulators for classification report
class_metrics = defaultdict(lambda: defaultdict(float))  # Stores precision/recall/F1 for each class

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")
    
    # Shuffle data
    train_data = train_data.sample(frac=1).reset_index(drop=True)
    test_data = test_data.sample(frac=1).reset_index(drop=True)
    
    # Replace infinities and fill NaN
    train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_data.fillna(train_data.mean(), inplace=True)
    test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_data.fillna(test_data.mean(), inplace=True)
    
    # Split features and labels
    train_y = train_data['label']
    train_X = train_data.drop(['label'], axis=1)
    test_y = test_data['label']
    test_X = test_data.drop(['label'], axis=1)
    
    # Convert to DMatrix for XGBoost
    dtrain = xgb.DMatrix(data=train_X, label=train_y)
    dtest = xgb.DMatrix(data=test_X, label=test_y)
    
    # Train the model
    bst = xgb.train(params, dtrain, num_round)
    
    # Make predictions
    test_probabilities = bst.predict(dtest)  # Outputs probabilities
    test_predictions = np.argmax(test_probabilities, axis=1)  # Convert to discrete class predictions
    
    # Calculate accuracy
    accuracy = accuracy_score(test_y, test_predictions)
    accuracy_scores.append(accuracy)
    
    # Calculate F2-Score
    f2_score = fbeta_score(test_y, test_predictions, beta=2, average='weighted')
    f2_scores.append(f2_score)
    
    # Calculate AUC-ROC (One-vs-Rest)
    auc_roc = roc_auc_score(test_y, test_probabilities, multi_class='ovr')
    auc_roc_scores.append(auc_roc)
    
    # Calculate AUC-PR (One-vs-Rest)
    auc_pr = average_precision_score(test_y, test_probabilities, average='weighted')
    auc_pr_scores.append(auc_pr)
    
    # Calculate MCC
    mcc = matthews_corrcoef(test_y, test_predictions)
    mcc_scores.append(mcc)
    
    # Classification Report
    report = classification_report(test_y, test_predictions, output_dict=True, zero_division=0)
    
    for class_label, metrics in report.items():
        if isinstance(metrics, dict):  # Skip non-class metrics
            for metric_name, metric_value in metrics.items():
                class_metrics[class_label][metric_name] += metric_value

# Calculate averages
average_accuracy = np.mean(accuracy_scores)
average_f2_score = np.mean(f2_scores)
average_auc_roc = np.mean(auc_roc_scores)
average_auc_pr = np.mean(auc_pr_scores)
average_mcc = np.mean(mcc_scores)

# Average the classification report metrics
average_class_metrics = {
    class_label: {metric_name: metric_value / num_runs for metric_name, metric_value in metrics.items()}
    for class_label, metrics in class_metrics.items()
}

# Output results
print(f"Average Test Accuracy: {average_accuracy:.4f}")
print(f"Average F2-Score (Weighted): {average_f2_score:.4f}")
print(f"Average AUC-ROC (One-vs-Rest): {average_auc_roc:.4f}")
print(f"Average AUC-PR: {average_auc_pr:.4f}")
print(f"Average Matthews Correlation Coefficient: {average_mcc:.4f}")

print("\nAverage Classification Report:")
for class_label, metrics in average_class_metrics.items():
    print(f"Class {class_label}:")
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.4f}")


Run 1/5
Run 2/5
Run 3/5
Run 4/5
Run 5/5
Average Test Accuracy: 0.9997
Average F2-Score (Weighted): 0.9997
Average AUC-ROC (One-vs-Rest): 1.0000
Average AUC-PR: 1.0000
Average Matthews Correlation Coefficient: 0.9993

Average Classification Report:
Class 0:
  precision: 0.9999
  recall: 1.0000
  f1-score: 0.9999
  support: 9117.0000
Class 1:
  precision: 1.0000
  recall: 1.0000
  f1-score: 1.0000
  support: 10.0000
Class 2:
  precision: 1.0000
  recall: 0.9706
  f1-score: 0.9851
  support: 34.0000
Class 3:
  precision: 0.9994
  recall: 0.9994
  f1-score: 0.9994
  support: 1781.0000
Class 4:
  precision: 0.9985
  recall: 1.0000
  f1-score: 0.9992
  support: 653.0000
Class 5:
  precision: 1.0000
  recall: 0.5000
  f1-score: 0.6667
  support: 2.0000
Class 6:
  precision: 1.0000
  recall: 1.0000
  f1-score: 1.0000
  support: 3.0000
Class macro avg:
  precision: 0.9997
  recall: 0.9243
  f1-score: 0.9501
  support: 11600.0000
Class weighted avg:
  precision: 0.9997
  recall: 0.9997
  f1-scor

Runtime: 32s

# Covertype

In [5]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import fbeta_score, roc_auc_score, average_precision_score, matthews_corrcoef
import numpy as np
from collections import defaultdict

# Load the training dataset
train_data = pd.read_csv("covtype_train.csv")
train_data['label'] = train_data['label'] - 1  # Subtract 1 to make labels zero-based

# Load the test dataset
test_data = pd.read_csv("covtype_test.csv")
test_data['label'] = test_data['label'] - 1  # Subtract 1 to make labels zero-based

# Parameters
num_runs = 5  # Number of iterations
num_round = 100
params = {
    "objective": "multi:softprob",  # Output probabilities
    "num_class": len(train_data['label'].unique()),  # Number of unique classes in the target
    "eval_metric": "mlogloss",
    "max_depth": 6,
    "eta": 0.3,
    "seed": 42
}

# Initialize accumulators
accuracy_scores = []
f2_scores = []
auc_roc_scores = []
auc_pr_scores = []
mcc_scores = []

# Initialize metrics accumulators for classification report
class_metrics = defaultdict(lambda: defaultdict(float))  # Stores precision/recall/F1 for each class

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")
    
    # Shuffle data
    train_data = train_data.sample(frac=1).reset_index(drop=True)
    test_data = test_data.sample(frac=1).reset_index(drop=True)
    
    # Replace infinities and fill NaN
    train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_data.fillna(train_data.mean(), inplace=True)
    test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_data.fillna(test_data.mean(), inplace=True)
    
    # Split features and labels
    train_y = train_data['label']
    train_X = train_data.drop(['label'], axis=1)
    test_y = test_data['label']
    test_X = test_data.drop(['label'], axis=1)
    
    # Convert to DMatrix for XGBoost
    dtrain = xgb.DMatrix(data=train_X, label=train_y)
    dtest = xgb.DMatrix(data=test_X, label=test_y)
    
    # Train the model
    bst = xgb.train(params, dtrain, num_round)
    
    # Make predictions
    test_probabilities = bst.predict(dtest)  # Outputs probabilities
    test_predictions = np.argmax(test_probabilities, axis=1)  # Convert to discrete class predictions
    
    # Calculate accuracy
    accuracy = accuracy_score(test_y, test_predictions)
    accuracy_scores.append(accuracy)
    
    # Calculate F2-Score
    f2_score = fbeta_score(test_y, test_predictions, beta=2, average='weighted')
    f2_scores.append(f2_score)
    
    # Calculate AUC-ROC (One-vs-Rest)
    auc_roc = roc_auc_score(test_y, test_probabilities, multi_class='ovr')
    auc_roc_scores.append(auc_roc)
    
    # Calculate AUC-PR (One-vs-Rest)
    auc_pr = average_precision_score(test_y, test_probabilities, average='weighted')
    auc_pr_scores.append(auc_pr)
    
    # Calculate MCC
    mcc = matthews_corrcoef(test_y, test_predictions)
    mcc_scores.append(mcc)
    
    # Classification Report
    report = classification_report(test_y, test_predictions, output_dict=True, zero_division=0)
    
    for class_label, metrics in report.items():
        if isinstance(metrics, dict):  # Skip non-class metrics
            for metric_name, metric_value in metrics.items():
                class_metrics[class_label][metric_name] += metric_value

# Calculate averages
average_accuracy = np.mean(accuracy_scores)
average_f2_score = np.mean(f2_scores)
average_auc_roc = np.mean(auc_roc_scores)
average_auc_pr = np.mean(auc_pr_scores)
average_mcc = np.mean(mcc_scores)

# Average the classification report metrics
average_class_metrics = {
    class_label: {metric_name: metric_value / num_runs for metric_name, metric_value in metrics.items()}
    for class_label, metrics in class_metrics.items()
}

# Output results
print(f"Average Test Accuracy: {average_accuracy:.4f}")
print(f"Average F2-Score (Weighted): {average_f2_score:.4f}")
print(f"Average AUC-ROC (One-vs-Rest): {average_auc_roc:.4f}")
print(f"Average AUC-PR: {average_auc_pr:.4f}")
print(f"Average Matthews Correlation Coefficient: {average_mcc:.4f}")

print("\nAverage Classification Report:")
for class_label, metrics in average_class_metrics.items():
    print(f"Class {class_label}:")
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.4f}")


Run 1/5
Run 2/5
Run 3/5
Run 4/5
Run 5/5
Average Test Accuracy: 0.8696
Average F2-Score (Weighted): 0.8692
Average AUC-ROC (One-vs-Rest): 0.9863
Average AUC-PR: 0.9457
Average Matthews Correlation Coefficient: 0.7895

Average Classification Report:
Class 0:
  precision: 0.8627
  recall: 0.8407
  f1-score: 0.8515
  support: 42368.0000
Class 1:
  precision: 0.8671
  recall: 0.8963
  f1-score: 0.8815
  support: 56661.0000
Class 2:
  precision: 0.8899
  recall: 0.9095
  f1-score: 0.8996
  support: 7151.0000
Class 3:
  precision: 0.8787
  recall: 0.8579
  f1-score: 0.8682
  support: 549.0000
Class 4:
  precision: 0.8891
  recall: 0.6077
  f1-score: 0.7219
  support: 1899.0000
Class 5:
  precision: 0.8493
  recall: 0.7970
  f1-score: 0.8223
  support: 3473.0000
Class 6:
  precision: 0.9508
  recall: 0.9139
  f1-score: 0.9320
  support: 4102.0000
Class macro avg:
  precision: 0.8839
  recall: 0.8319
  f1-score: 0.8539
  support: 116203.0000
Class weighted avg:
  precision: 0.8697
  recall: 0.8

Runtime: 41.6s

# KDD

In [6]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import fbeta_score, roc_auc_score, average_precision_score, matthews_corrcoef
import numpy as np
from collections import defaultdict

# Define label mapping for KDD dataset labels
label_mapping = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

# Load the training dataset
train_data = pd.read_csv("kdd_train.csv")
train_data['label'] = train_data['label'].map(label_mapping)  # Convert labels to numeric values

# Load the test dataset
test_data = pd.read_csv("kdd_test.csv")
test_data['label'] = test_data['label'].map(label_mapping)  # Convert labels to numeric values

# Parameters
num_runs = 5  # Number of iterations
num_round = 100
params = {
    "objective": "multi:softprob",  # Output probabilities
    "num_class": len(train_data['label'].unique()),  # Number of unique classes in the target
    "eval_metric": "mlogloss",
    "max_depth": 6,
    "eta": 0.3,
    "seed": 42
}

# Initialize accumulators
accuracy_scores = []
f2_scores = []
auc_roc_scores = []
auc_pr_scores = []
mcc_scores = []

# Initialize metrics accumulators for classification report
class_metrics = defaultdict(lambda: defaultdict(float))  # Stores precision/recall/F1 for each class

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")
    
    # Shuffle data
    train_data = train_data.sample(frac=1).reset_index(drop=True)
    test_data = test_data.sample(frac=1).reset_index(drop=True)
    
    # Replace infinities and fill NaN
    train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_data.fillna(train_data.mean(), inplace=True)
    test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_data.fillna(test_data.mean(), inplace=True)
    
    # Split features and labels
    train_y = train_data['label']
    train_X = train_data.drop(['label'], axis=1)
    test_y = test_data['label']
    test_X = test_data.drop(['label'], axis=1)
    
    # Convert to DMatrix for XGBoost
    dtrain = xgb.DMatrix(data=train_X, label=train_y)
    dtest = xgb.DMatrix(data=test_X, label=test_y)
    
    # Train the model
    bst = xgb.train(params, dtrain, num_round)
    
    # Make predictions
    test_probabilities = bst.predict(dtest)  # Outputs probabilities
    test_predictions = np.argmax(test_probabilities, axis=1)  # Convert to discrete class predictions
    
    # Calculate accuracy
    accuracy = accuracy_score(test_y, test_predictions)
    accuracy_scores.append(accuracy)
    
    # Calculate F2-Score
    f2_score = fbeta_score(test_y, test_predictions, beta=2, average='weighted')
    f2_scores.append(f2_score)
    
    # Calculate AUC-ROC (One-vs-Rest)
    auc_roc = roc_auc_score(test_y, test_probabilities, multi_class='ovr')
    auc_roc_scores.append(auc_roc)
    
    # Calculate AUC-PR (One-vs-Rest)
    auc_pr = average_precision_score(test_y, test_probabilities, average='weighted')
    auc_pr_scores.append(auc_pr)
    
    # Calculate MCC
    mcc = matthews_corrcoef(test_y, test_predictions)
    mcc_scores.append(mcc)
    
    # Classification Report
    report = classification_report(test_y, test_predictions, output_dict=True, zero_division=0)
    
    for class_label, metrics in report.items():
        if isinstance(metrics, dict):  # Skip non-class metrics
            for metric_name, metric_value in metrics.items():
                class_metrics[class_label][metric_name] += metric_value

# Calculate averages
average_accuracy = np.mean(accuracy_scores)
average_f2_score = np.mean(f2_scores)
average_auc_roc = np.mean(auc_roc_scores)
average_auc_pr = np.mean(auc_pr_scores)
average_mcc = np.mean(mcc_scores)

# Average the classification report metrics
average_class_metrics = {
    class_label: {metric_name: metric_value / num_runs for metric_name, metric_value in metrics.items()}
    for class_label, metrics in class_metrics.items()
}

# Output results
print(f"Average Test Accuracy: {average_accuracy:.4f}")
print(f"Average F2-Score (Weighted): {average_f2_score:.4f}")
print(f"Average AUC-ROC (One-vs-Rest): {average_auc_roc:.4f}")
print(f"Average AUC-PR: {average_auc_pr:.4f}")
print(f"Average Matthews Correlation Coefficient: {average_mcc:.4f}")

print("\nAverage Classification Report:")
for class_label, metrics in average_class_metrics.items():
    print(f"Class {class_label}:")
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.4f}")


Run 1/5
Run 2/5
Run 3/5
Run 4/5
Run 5/5
Average Test Accuracy: 0.9998
Average F2-Score (Weighted): 0.9998
Average AUC-ROC (One-vs-Rest): 1.0000
Average AUC-PR: 1.0000
Average Matthews Correlation Coefficient: 0.9980

Average Classification Report:
Class 0:
  precision: 0.9999
  recall: 1.0000
  f1-score: 0.9999
  support: 194557.0000
Class 1:
  precision: 0.9987
  recall: 0.9987
  f1-score: 0.9987
  support: 3178.0000
Class 2:
  precision: 0.9976
  recall: 0.9996
  f1-score: 0.9986
  support: 2496.0000
Class 3:
  precision: 0.9995
  recall: 0.9976
  f1-score: 0.9986
  support: 2083.0000
Class 4:
  precision: 1.0000
  recall: 0.9935
  f1-score: 0.9967
  support: 463.0000
Class 5:
  precision: 0.9977
  recall: 0.9977
  f1-score: 0.9977
  support: 441.0000
Class 6:
  precision: 0.9950
  recall: 0.9755
  f1-score: 0.9851
  support: 204.0000
Class 7:
  precision: 1.0000
  recall: 1.0000
  f1-score: 1.0000
  support: 196.0000
Class 8:
  precision: 1.0000
  recall: 0.9811
  f1-score: 0.9905
 

Runtime: 3m 37.5s

# Darknet

In [7]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import fbeta_score, roc_auc_score, average_precision_score, matthews_corrcoef
import numpy as np
from collections import defaultdict

# Define label mapping for Darknet dataset labels
label_mapping = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

# Load the training dataset
train_data = pd.read_csv("darknet_train.csv")
train_data['label'] = train_data['label'].map(label_mapping)  # Convert labels to numeric values

# Load the test dataset
test_data = pd.read_csv("darknet_test.csv")
test_data['label'] = test_data['label'].map(label_mapping)  # Convert labels to numeric values

# Parameters
num_runs = 5  # Number of iterations
num_round = 100
params = {
    "objective": "multi:softprob",  # Output probabilities
    "num_class": len(train_data['label'].unique()),  # Number of unique classes in the target
    "eval_metric": "mlogloss",
    "max_depth": 6,
    "eta": 0.3,
    "seed": 42
}

# Initialize accumulators
accuracy_scores = []
f2_scores = []
auc_roc_scores = []
auc_pr_scores = []
mcc_scores = []

# Initialize metrics accumulators for classification report
class_metrics = defaultdict(lambda: defaultdict(float))  # Stores precision/recall/F1 for each class

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")
    
    # Shuffle data
    train_data = train_data.sample(frac=1).reset_index(drop=True)
    test_data = test_data.sample(frac=1).reset_index(drop=True)
    
    # Replace infinities and fill NaN
    train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_data.fillna(train_data.mean(), inplace=True)
    test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_data.fillna(test_data.mean(), inplace=True)
    
    # Split features and labels
    train_y = train_data['label']
    train_X = train_data.drop(['label'], axis=1)
    test_y = test_data['label']
    test_X = test_data.drop(['label'], axis=1)
    
    # Convert to DMatrix for XGBoost
    dtrain = xgb.DMatrix(data=train_X, label=train_y)
    dtest = xgb.DMatrix(data=test_X, label=test_y)
    
    # Train the model
    bst = xgb.train(params, dtrain, num_round)
    
    # Make predictions
    test_probabilities = bst.predict(dtest)  # Outputs probabilities
    test_predictions = np.argmax(test_probabilities, axis=1)  # Convert to discrete class predictions
    
    # Calculate accuracy
    accuracy = accuracy_score(test_y, test_predictions)
    accuracy_scores.append(accuracy)
    
    # Calculate F2-Score
    f2_score = fbeta_score(test_y, test_predictions, beta=2, average='weighted')
    f2_scores.append(f2_score)
    
    # Calculate AUC-ROC (One-vs-Rest)
    auc_roc = roc_auc_score(test_y, test_probabilities, multi_class='ovr')
    auc_roc_scores.append(auc_roc)
    
    # Calculate AUC-PR (One-vs-Rest)
    auc_pr = average_precision_score(test_y, test_probabilities, average='weighted')
    auc_pr_scores.append(auc_pr)
    
    # Calculate MCC
    mcc = matthews_corrcoef(test_y, test_predictions)
    mcc_scores.append(mcc)
    
    # Classification Report
    report = classification_report(test_y, test_predictions, output_dict=True, zero_division=0)
    
    for class_label, metrics in report.items():
        if isinstance(metrics, dict):  # Skip non-class metrics
            for metric_name, metric_value in metrics.items():
                class_metrics[class_label][metric_name] += metric_value

# Calculate averages
average_accuracy = np.mean(accuracy_scores)
average_f2_score = np.mean(f2_scores)
average_auc_roc = np.mean(auc_roc_scores)
average_auc_pr = np.mean(auc_pr_scores)
average_mcc = np.mean(mcc_scores)

# Average the classification report metrics
average_class_metrics = {
    class_label: {metric_name: metric_value / num_runs for metric_name, metric_value in metrics.items()}
    for class_label, metrics in class_metrics.items()
}

# Output results
print(f"Average Test Accuracy: {average_accuracy:.4f}")
print(f"Average F2-Score (Weighted): {average_f2_score:.4f}")
print(f"Average AUC-ROC (One-vs-Rest): {average_auc_roc:.4f}")
print(f"Average AUC-PR: {average_auc_pr:.4f}")
print(f"Average Matthews Correlation Coefficient: {average_mcc:.4f}")

print("\nAverage Classification Report:")
for class_label, metrics in average_class_metrics.items():
    print(f"Class {class_label}:")
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.4f}")


Run 1/5
Run 2/5
Run 3/5
Run 4/5
Run 5/5
Average Test Accuracy: 0.9997
Average F2-Score (Weighted): 0.9997
Average AUC-ROC (One-vs-Rest): 1.0000
Average AUC-PR: 0.9999
Average Matthews Correlation Coefficient: 0.9991

Average Classification Report:
Class 0:
  precision: 0.9999
  recall: 1.0000
  f1-score: 0.9999
  support: 26862.0000
Class 1:
  precision: 0.9998
  recall: 1.0000
  f1-score: 0.9999
  support: 2657.0000
Class 2:
  precision: 1.0000
  recall: 0.9978
  f1-score: 0.9989
  support: 908.0000
Class 3:
  precision: 1.0000
  recall: 0.9977
  f1-score: 0.9988
  support: 522.0000
Class 4:
  precision: 0.9932
  recall: 1.0000
  f1-score: 0.9966
  support: 293.0000
Class 5:
  precision: 1.0000
  recall: 1.0000
  f1-score: 1.0000
  support: 269.0000
Class 6:
  precision: 1.0000
  recall: 1.0000
  f1-score: 1.0000
  support: 116.0000
Class 7:
  precision: 0.9600
  recall: 0.9057
  f1-score: 0.9320
  support: 53.0000
Class 8:
  precision: 1.0000
  recall: 1.0000
  f1-score: 1.0000
  sup

Runtime: 20.4s